In [ ]:
db = 'gthruth_valeflores_v3'

tschema = [{
    'TABLE' : 'gthruth_fishnet',
    'COLUMNS' : [
        'cid INT PRIMARY KEY',
        'level INT NOT NULL',
        'rootid INT',
        'geom geometry(POLYGON, 32629) NOT NULL'
    ],
    "IDXS" : {
        'cellgeom_idx' : 'gist (geom)'
    },
    "CONSTRAINTS" : [
        "FOREIGN KEY (rootid) REFERENCES gthruth_fishnet (cid)"
    ],
    "DATA" : '/mnt/d/autocls_exp3/groundtruth/truthground_valeflores.shp',
    "MAPCOLS" : {'cid' : 'cellid', 'geom' : 'geometry', 'level' : 'lvl'}
}, {
    'TABLE' : 'refmaps',
    'COLUMNS' : [
        'mapid INT PRIMARY KEY',
        "slug TEXT NOT NULL",
        'name TEXT NOT NULL',
        'year INTEGER NOT NULL'
    ],
    "DATA" : ["/mnt/d/autocls_exp3/lulc_products.xlsx", "lulc_maps"],
    "MAPCOLS" : {
        "mapid" : "did",
        "slug"  : "slug",
        "name"  : "name",
        "year"  : "year"
    }
}, {
    'TABLE' : 'mapclasses',
    'COLUMNS' : [
        'classid INT PRIMARY KEY',
        'code TEXT NOT NULL',
        'classname TEXT NOT NULL',
        'level INT',
        'mapfk INT NOT NULL'
    ],
    "CONSTRAINTS" : [
        "FOREIGN KEY (mapfk) REFERENCES refmaps (mapid)"
    ],
    "DATA" : ["/mnt/d/autocls_exp3/lulc_products.xlsx", "lulc_classes"],
    "MAPCOLS" : {
        'classid' : '_id',
        'code'    : 'code',
        'classname' : 'class',
        'level' : 'level',
        'mapfk' : 'mapid'
    }
}, {
    'TABLE' : 'pxclasses',
    'COLUMNS' : [
        'pxid INT',
        'clsid INT'
    ],
    "CONSTRAINTS" : [
        'PRIMARY KEY (pxid, clsid)',
        'FOREIGN KEY (pxid) REFERENCES gthruth_fishnet (cid)',
        'FOREIGN KEY (clsid) REFERENCES mapclasses (classid)'
    ]
}, {
    'TABLE' : 'experiments',
    'COLUMNS' : [
        'eid INT PRIMARY KEY',
        'slug TEXT NOT NULL',
        'description TEXT NOT NULL'
    ]
}, {
    'TABLE' : 'expclasses',
    'COLUMNS' : [
        'expclsid INT PRIMARY KEY',
        'codeval INT NOT NULL',
        'nameval TEXT NOT NULL',
        'expfk INT NOT NULL'
    ],
    "CONSTRAINTS" : [
        "FOREIGN KEY (expfk) REFERENCES experiments (eid)
    ]
}, {
    'TABLE' : 'pxexpclasses',
    'COLUMNS': [
        'pxid INT',
        'eclsid INT'
    ],
    'CONSTRAINTS' : [
        'PRIMARY KEY (pxid, eclsid)',
        'FOREIGN KEY (pxid) REFERENCES gthruth_fishnet (cid)',
        'FOREIGN KEY (eclsid) REFERENCES expclasses (expclsid)'
    ]
}]

In [ ]:
from glass.sql.db import create_pgdb
from glass.prop.df import is_shp
from glass.it.db import shp_to_psql, tbl_to_db

In [ ]:
# Create database and tables

ndb = create_pgdb(db, overwrite=True, schema=tschema)

In [ ]:
# Send data to the tables
for t in tschema:
    if "DATA" in t:
        if type(t["DATA"]) == list and len(t["DATA"]) > 1:
            dfile, slyr = t["DATA"]
        else:
            dfile, slyr = t["DATA"], None

        # Check if file is a Vetor file
        if is_shp(dfile):
            # Send geospatial data
            shp_to_psql(
                ndb, dfile,
                tnames={dfile : t["TABLE"]},
                map_cols={t["MAPCOLS"][k] : k for k in t["MAPCOLS"]},
                mantain_map_cols=True
            )

        else:
            tbl_to_db(
                dfile, ndb, t["TABLE"], sheet=slyr,
                colsMap={t["MAPCOLS"][k] : k for k in t["MAPCOLS"]},
                isAppend=True
            )